<a href="https://colab.research.google.com/github/YZabolotnii/AB_test_py/blob/main/AB_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install google-cloud-bigquery

from google.colab import auth
from google.cloud import bigquery
import pandas as pd
import numpy as np
import statsmodels.api as sm

auth.authenticate_user()
client = bigquery.Client(project="data-analytics-mate")

In [ ]:
query = """
 WITH session_info AS (
 SELECT
   s.date,
   s.ga_session_id,
   sp.country,
   sp.device,
   sp.continent,
   sp.channel,
   ab.test,
   ab.test_group
 FROM
   `DA.ab_test` ab
 JOIN
   `DA.session` s
 ON
   ab.ga_session_id = s.ga_session_id
 JOIN
   `DA.session_params` sp
 ON
   sp.ga_session_id = ab.ga_session_id ),
 session_with_orders AS (
 SELECT
   session_info.date,
   session_info.country,
   session_info.device,
   session_info.continent,
   session_info.channel,
   session_info.test,
   session_info.test_group,
   COUNT(DISTINCT o.ga_session_id) AS session_with_orders
 FROM
   `DA.order` o
 JOIN
   session_info
 ON
   o.ga_session_id = session_info.ga_session_id
 GROUP BY
   session_info.date,
   session_info.country,
   session_info.device,
   session_info.continent,
   session_info.channel,
   session_info.test,
   session_info.test_group),
 events AS (
 SELECT
   session_info.date,
   session_info.country,
   session_info.device,
   session_info.continent,
   session_info.channel,
   session_info.test,
   session_info.test_group,
   ep.event_name,
   COUNT(ep.ga_session_id) AS event_cnt
 FROM
   `DA.event_params` ep
 JOIN
   session_info
 ON
   ep.ga_session_id = session_info.ga_session_id
 GROUP BY
   session_info.date,
   session_info.country,
   session_info.device,
   session_info.continent,
   session_info.channel,
   session_info.test,
   session_info.test_group,
   ep.event_name ),
 session AS (
 SELECT
   session_info.date,
   session_info.country,
   session_info.device,
   session_info.continent,
   session_info.channel,
   session_info.test,
   session_info.test_group,
   COUNT(DISTINCT session_info.ga_session_id) AS session_cnt
 FROM
   session_info
 GROUP BY
   session_info.date,
   session_info.country,
   session_info.device,
   session_info.continent,
   session_info.channel,
   session_info.test,
   session_info.test_group ),
 account AS (
 SELECT
   session_info.date,
   session_info.country,
   session_info.device,
   session_info.continent,
   session_info.channel,
   session_info.test,
   session_info.test_group,
   COUNT(DISTINCT acs.ga_session_id) AS new_account_cnt
 FROM
   `DA.account_session` acs
 JOIN
   session_info
 ON
   acs.ga_session_id = session_info.ga_session_id
 GROUP BY
   session_info.date,
   session_info.country,
   session_info.device,
   session_info.continent,
   session_info.channel,
   session_info.test,
   session_info.test_group )
SELECT
 session_with_orders.date,
 session_with_orders.country,
 session_with_orders.device,
 session_with_orders.continent,
 session_with_orders.channel,
 session_with_orders.test,
 session_with_orders.test_group,
 'session_with_orders' AS event_name,
 session_with_orders.session_with_orders AS value
FROM
 session_with_orders
UNION ALL
SELECT
 events.date,
 events.country,
 events.device,
 events.continent,
 events.channel,
 events.test,
 events.test_group,
 events.event_name,
 events.event_cnt AS value
FROM
 events
UNION ALL
SELECT
 session.date,
 session.country,
 session.device,
 session.continent,
 session.channel,
 session.test,
 session.test_group,
 'session' AS event_name,
 session.session_cnt AS value
FROM
 session
UNION ALL
SELECT
 account.date,
 account.country,
 account.device,
 account.continent,
 account.channel,
 account.test,
 account.test_group,
 'new account' AS event_name,
 account.new_account_cnt AS value
FROM
 account
"""

query_job = client.query(query)
results = query_job.result()

df = results.to_dataframe()
df.head()

,date,country,device,continent,channel,test,test_group,event_name,value
0,2020-11-01,Ecuador,desktop,Americas,Organic Search,2,2,session_with_orders,1
1,2020-11-02,Trinidad & Tobago,desktop,Americas,Organic Search,2,2,session_with_orders,1
2,2020-11-03,Cambodia,desktop,Asia,Direct,2,1,session_with_orders,1
3,2020-11-03,Paraguay,desktop,Americas,Direct,2,2,session_with_orders,1
4,2020-11-04,Kazakhstan,desktop,Asia,Undefined,2,1,session_with_orders,1


In [ ]:
from scipy.stats import norm

# Формули метрик: подія / session
metrics = {
    "add_payment_info": "add_payment_info / session",
    "add_shipping_info": "add_shipping_info / session",
    "begin_checkout": "begin_checkout / session",
    "new account": "new account / session"
}

results = []

# Обробка кожного A/B тесту окремо
for test_id in df["test"].unique():
    test_df = df[df["test"] == test_id]

    control = test_df[test_df["test_group"] == 1]
    experiment = test_df[test_df["test_group"] == 2]

    if control.empty or experiment.empty:
        print(f"[!] Пропущено тест '{test_id}': відсутні дані в одній з груп.")
        continue

    for metric, formula in metrics.items():
        try:
            # Сума подій для контрольної групи
            num_control = control[control["event_name"] == metric]["value"].sum()
            den_control = control[control["event_name"] == "session"]["value"].sum()

            # Сума подій для тестової групи
            num_experiment = experiment[experiment["event_name"] == metric]["value"].sum()
            den_experiment = experiment[experiment["event_name"] == "session"]["value"].sum()

            # Перевірка нульового знаменника
            if den_control == 0 or den_experiment == 0:
                print(f"[!] Нульовий знаменник у метриці '{metric}' для тесту '{test_id}'")
                continue

            # Розрахунок конверсій
            conv_rate_A = num_control / den_control
            conv_rate_B = num_experiment / den_experiment
            change = (conv_rate_B - conv_rate_A) / conv_rate_A * 100

            # Об’єднана ймовірність
            pooled_p = (num_control + num_experiment) / (den_control + den_experiment)

            # Стандартна помилка
            std_err = np.sqrt(pooled_p * (1 - pooled_p) * (1 / den_control + 1 / den_experiment))

            # Z-статистика та p-value
            z = (conv_rate_B - conv_rate_A) / std_err
            p = 2 * (1 - norm.cdf(abs(z)))

            results.append({
                "test_number": test_id,
                "metric_formula": formula,
                "numerator_event": metric,
                "denominator_event": "session",
                "num_control": num_control,
                "den_control": den_control,
                "conversion_rate_A": round(conv_rate_A, 10),
                "num_experiment": num_experiment,
                "den_experiment": den_experiment,
                "conversion_rate_B": round(conv_rate_B, 10),
                "metric_change_percent": round(change, 10),
                "z_stat": round(z, 10),
                "p_value": round(p, 10),
                "significant": p < 0.05
            })

        except Exception as err:
            print(f"[!] Помилка для метрики '{metric}' у тесті '{test_id}': {err}")

# Підсумкова таблиця
results_df = pd.DataFrame(results)
results_df


,test_number,metric_formula,numerator_event,denominator_event,num_control,den_control,conversion_rate_A,num_experiment,den_experiment,conversion_rate_B,metric_change_percent,z_stat,p_value,significant
0,2,add_payment_info / session,add_payment_info,session,2344,50637,0.046290,2409,50244,0.047946,3.576911,1.240994,0.214608,False
1,2,add_shipping_info / session,add_shipping_info,session,3480,50637,0.068724,3510,50244,0.069859,1.650995,0.709557,0.477979,False
2,2,begin_checkout / session,begin_checkout,session,4262,50637,0.084168,4313,50244,0.085841,1.988164,0.952898,0.340642,False
3,2,new account / session,new account,session,4165,50637,0.082252,4184,50244,0.083274,1.241934,0.588793,0.556000,False
4,1,add_payment_info / session,add_payment_info,session,1988,45362,0.043825,2229,45193,0.049322,12.542021,3.924884,0.000087,True
5,1,add_shipping_info / session,add_shipping_info,session,3034,45362,0.066884,3221,45193,0.071272,6.560481,2.603571,0.009226,True
6,1,begin_checkout / session,begin_checkout,session,3784,45362,0.083418,4021,45193,0.088974,6.660587,2.978783,0.002894,True
7,1,new account / session,new account,session,3823,45362,0.084278,3681,45193,0.081451,-3.354299,-1.542883,0.122859,False
8,4,add_payment_info / session,add_payment_info,session,3731,105079,0.035507,3601,105141,0.034249,-3.541234,-1.571106,0.116158,False
9,4,add_shipping_info / session,add_shipping_info,session,5128,105079,0.048801,4956,105141,0.047137,-3.411125,-1.785795,0.074132,False


In [ ]:
from google.colab import drive

drive.mount('/content/drive')
results_df.to_excel('/content/drive/MyDrive/MA_papka/ab_results.xlsx', index=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Final [file](https://docs.google.com/spreadsheets/d/1Vya7eYVaHJeA5rn7X5MCBuGsasFlh1qb/edit?gid=1593775369#gid=1593775369)

Tableau [Dashboard](https://public.tableau.com/app/profile/yaroslav.zabolotnii/viz/A_Btest_17511246280000/ABtest)